## Ologit test
based on https://www.statsmodels.org/devel/examples/notebooks/generated/ordinal_regression.html

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats

from statsmodels.miscmodels.ordinal_model import OrderedModel

In [2]:
df = pd.read_stata("https://stats.idre.ucla.edu/stat/data/ologit.dta")

In [3]:
df.head()

,apply,pared,public,gpa
0,very likely,0,0,3.26
1,somewhat likely,1,0,3.21
2,unlikely,1,1,3.94
3,somewhat likely,0,0,2.81
4,somewhat likely,0,0,2.53


In [4]:
df.dtypes

apply     category
pared         int8
public        int8
gpa        float32
dtype: object

In [5]:
df["apply"].dtype

CategoricalDtype(categories=['unlikely', 'somewhat likely', 'very likely'], ordered=True, categories_dtype=object)

In [6]:
mod_log = OrderedModel(df["apply"], df[["pared", "public", "gpa"]], distr="logit")

res_log = mod_log.fit(method='bfgs', disp=False)
res_log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                  apply   Log-Likelihood:                -358.51
Model:                   OrderedModel   AIC:                             727.0
Method:            Maximum Likelihood   BIC:                             747.0
Date:                Tue, 17 Dec 2024                                         
Time:                        17:24:06                                         
No. Observations:                 400                                         
Df Residuals:                     395                                         
Df Model:                           3                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
pared                           1.0476      0.266      3.942      0.000       0.527       1.569
public                         -0.0586      0.298     -0.197      0.844      -0.642       0.525
gpa                             0.6158      0.261      2.363      0.018       0.105       1.127
unlikely/somewhat likely        2.2035      0.780      2.827      0.005       0.676       3.731
somewhat likely/very likely     0.7398      0.080      9.236      0.000       0.583       0.897
===============================================================================================
"""

In [19]:
summary = res_log.summary().tables[1]
summary = pd.DataFrame(summary.data[1:], columns=summary.data[0])
summary = summary.astype({"coef":"float64","std err":"float64", "z":"float64", "[0.025":"float64", "0.975]":"float64"})
summary

,,coef,std err,z,P>|z|,[0.025,0.975]
0,pared,1.0476,0.266,3.942,0.000,0.527,1.569
1,public,-0.0586,0.298,-0.197,0.844,-0.642,0.525
2,gpa,0.6158,0.261,2.363,0.018,0.105,1.127
3,unlikely/somewhat likely,2.2035,0.780,2.827,0.005,0.676,3.731
4,somewhat likely/very likely,0.7398,0.080,9.236,0.000,0.583,0.897


In [20]:
summary['z']

0    3.942
1   -0.197
2    2.363
3    2.827
4    9.236
Name: z, dtype: float64

In [21]:
z_values = summary['z']
p_values = 2 * (1 - stats.norm.cdf(abs(z_values)))

In [23]:
summary['p'] = p_values
summary

,,coef,std err,z,P>|z|,[0.025,0.975],p
0,pared,1.0476,0.266,3.942,0.000,0.527,1.569,0.000081
1,public,-0.0586,0.298,-0.197,0.844,-0.642,0.525,0.843828
2,gpa,0.6158,0.261,2.363,0.018,0.105,1.127,0.018128
3,unlikely/somewhat likely,2.2035,0.780,2.827,0.005,0.676,3.731,0.004699
4,somewhat likely/very likely,0.7398,0.080,9.236,0.000,0.583,0.897,0.000000
